# Statistical Analysis of Stem Cell Therapy Clinical Trials

## Research Hour Documentation - Statistical Analysis Session

**Date**: Current Session  
**Objective**: Perform comprehensive statistical analysis on real clinical trial data for stem cell therapy in epilepsy and diabetes  
**Data Sources**: ClinicalTrials.gov, published meta-analyses, FDA submissions  


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_ind, chi2_contingency, mannwhitneyu
import statsmodels.api as sm
from statsmodels.stats.meta_analysis import combine_effects
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 1. Data Loading and Initial Exploration

In [ ]:
# Load clinical trial data
df = pd.read_csv('../data/clinical_trial_data.csv')

print("Dataset Overview:")
print(f"Total trials: {len(df)}")
print(f"Conditions studied: {df['condition'].unique()}")
print(f"Interventions: {df['intervention'].nunique()}")
print(f"Total patients: {df['n_patients'].sum()}")

# Display basic statistics
df.head()

In [ ]:
# Data quality assessment
print("Data Quality Assessment:")
print(f"Missing values:")
print(df.isnull().sum())
print(f"\nData types:")
print(df.dtypes)

## 2. Descriptive Statistics by Condition

In [ ]:
# Separate data by condition
epilepsy_data = df[df['condition'] == 'Epilepsy'].copy()
diabetes_t1_data = df[df['condition'] == 'Type_1_Diabetes'].copy()
diabetes_t2_data = df[df['condition'] == 'Type_2_Diabetes'].copy()

print("=== EPILEPSY TRIALS ANALYSIS ===")
print(f"Number of trials: {len(epilepsy_data)}")
print(f"Total patients: {epilepsy_data['n_patients'].sum()}")
print(f"Mean seizure reduction: {epilepsy_data[epilepsy_data['primary_endpoint'] == 'seizure_reduction_percent']['endpoint_value'].mean():.1f}%")
print(f"Range: {epilepsy_data[epilepsy_data['primary_endpoint'] == 'seizure_reduction_percent']['endpoint_value'].min():.1f}% - {epilepsy_data[epilepsy_data['primary_endpoint'] == 'seizure_reduction_percent']['endpoint_value'].max():.1f}%")

print("\n=== TYPE 1 DIABETES TRIALS ANALYSIS ===")
print(f"Number of trials: {len(diabetes_t1_data)}")
print(f"Total patients: {diabetes_t1_data['n_patients'].sum()}")

# Insulin independence rates
insulin_indep = diabetes_t1_data[diabetes_t1_data['primary_endpoint'] == 'insulin_independence_rate']['endpoint_value']
if len(insulin_indep) > 0:
    print(f"Mean insulin independence rate: {insulin_indep.mean():.1f}%")
    print(f"Range: {insulin_indep.min():.1f}% - {insulin_indep.max():.1f}%")

print("\n=== TYPE 2 DIABETES TRIALS ANALYSIS ===")
print(f"Number of trials: {len(diabetes_t2_data)}")
print(f"Total patients: {diabetes_t2_data['n_patients'].sum()}")

## 3. Statistical Hypothesis Testing

In [ ]:
# Test 1: Is there a significant difference in efficacy between epilepsy and diabetes trials?
print("=== HYPOTHESIS TEST 1: Efficacy Comparison Across Conditions ===")

# For epilepsy: seizure reduction percentages
epilepsy_outcomes = epilepsy_data[epilepsy_data['primary_endpoint'] == 'seizure_reduction_percent']['endpoint_value']

# For diabetes: insulin independence rates
diabetes_outcomes = diabetes_t1_data[diabetes_t1_data['primary_endpoint'] == 'insulin_independence_rate']['endpoint_value']

if len(epilepsy_outcomes) > 1 and len(diabetes_outcomes) > 1:
    # Mann-Whitney U test (non-parametric)
    statistic, p_value = mannwhitneyu(epilepsy_outcomes, diabetes_outcomes, alternative='two-sided')
    print(f"Mann-Whitney U test:")
    print(f"Epilepsy median efficacy: {epilepsy_outcomes.median():.1f}%")
    print(f"Diabetes median efficacy: {diabetes_outcomes.median():.1f}%")
    print(f"U-statistic: {statistic:.3f}")
    print(f"p-value: {p_value:.6f}")
    print(f"Significant difference: {'Yes' if p_value < 0.05 else 'No'}")
else:
    print("Insufficient data for comparison test")

In [ ]:
# Test 2: Treatment duration effect analysis
print("\n=== HYPOTHESIS TEST 2: Treatment Duration vs. Efficacy ===")

# Filter data with both follow-up and efficacy measures
duration_efficacy = df[(df['follow_up_months'].notna()) & 
                      (df['endpoint_value'].notna()) &
                      (df['primary_endpoint'].isin(['seizure_reduction_percent', 'insulin_independence_rate']))].copy()

if len(duration_efficacy) > 3:
    # Correlation analysis
    correlation_coef, correlation_p = stats.pearsonr(duration_efficacy['follow_up_months'], 
                                                   duration_efficacy['endpoint_value'])
    print(f"Pearson correlation between follow-up duration and efficacy:")
    print(f"Correlation coefficient: {correlation_coef:.3f}")
    print(f"p-value: {correlation_p:.6f}")
    print(f"Interpretation: {'Significant positive correlation' if correlation_p < 0.05 and correlation_coef > 0 else 'No significant correlation'}")
    
    # Linear regression
    X = sm.add_constant(duration_efficacy['follow_up_months'])
    y = duration_efficacy['endpoint_value']
    model = sm.OLS(y, X).fit()
    print(f"\nLinear Regression Results:")
    print(f"R-squared: {model.rsquared:.3f}")
    print(f"Slope: {model.params[1]:.3f} (efficacy change per month)")
    print(f"Slope p-value: {model.pvalues[1]:.6f}")
else:
    print("Insufficient data for duration analysis")

## 4. Meta-Analysis of Diabetes Outcomes

In [ ]:
# Meta-analysis using available confidence intervals
print("=== META-ANALYSIS: DIABETES HbA1c REDUCTION ===")

# Extract meta-analysis data with confidence intervals
meta_data = df[(df['phase'] == 'Meta_Analysis') & 
               (df['primary_endpoint'] == 'hba1c_change_absolute') &
               (df['ci_lower'].notna()) & 
               (df['ci_upper'].notna())].copy()

if len(meta_data) >= 2:
    print("Available meta-analysis data:")
    for idx, row in meta_data.iterrows():
        print(f"{row['condition']}: Effect = {row['endpoint_value']:.2f}, CI = [{row['ci_lower']:.2f}, {row['ci_upper']:.2f}], N = {row['n_patients']}")
    
    # Calculate pooled effect size
    effects = meta_data['endpoint_value'].values
    variances = ((meta_data['ci_upper'] - meta_data['ci_lower']) / (2 * 1.96)) ** 2
    weights = 1 / variances
    
    pooled_effect = np.sum(effects * weights) / np.sum(weights)
    pooled_variance = 1 / np.sum(weights)
    pooled_se = np.sqrt(pooled_variance)
    
    # 95% confidence interval for pooled effect
    pooled_ci_lower = pooled_effect - 1.96 * pooled_se
    pooled_ci_upper = pooled_effect + 1.96 * pooled_se
    
    print(f"\nPooled Meta-Analysis Results:")
    print(f"Pooled HbA1c reduction: {pooled_effect:.2f}% (95% CI: {pooled_ci_lower:.2f}, {pooled_ci_upper:.2f})")
    print(f"Total patients: {meta_data['n_patients'].sum()}")
    
    # Test for heterogeneity (Q-statistic)
    Q = np.sum(weights * (effects - pooled_effect) ** 2)
    df_q = len(effects) - 1
    p_heterogeneity = 1 - stats.chi2.cdf(Q, df_q)
    I_squared = max(0, (Q - df_q) / Q) * 100
    
    print(f"Heterogeneity analysis:")
    print(f"Q-statistic: {Q:.3f}, df = {df_q}, p = {p_heterogeneity:.6f}")
    print(f"I² = {I_squared:.1f}% ({'Low' if I_squared < 25 else 'Moderate' if I_squared < 75 else 'High'} heterogeneity)")
else:
    print("Insufficient meta-analysis data available")

## 5. Survival Analysis Simulation

In [ ]:
# Simulate treatment durability based on available data
print("=== TREATMENT DURABILITY ANALYSIS ===")

# Extract duration data by intervention type
duration_data = df[df['follow_up_months'].notna()].groupby('intervention').agg({
    'follow_up_months': ['mean', 'median', 'std'],
    'endpoint_value': 'mean',
    'n_patients': 'sum'
}).round(2)

print("Treatment Duration by Intervention Type:")
print(duration_data)

# Calculate treatment persistence rates
print("\nTreatment Persistence Analysis:")
for intervention in df['intervention'].unique():
    if pd.notna(intervention):
        intervention_data = df[df['intervention'] == intervention]
        if len(intervention_data) > 0:
            max_followup = intervention_data['follow_up_months'].max()
            mean_efficacy = intervention_data['endpoint_value'].mean()
            print(f"{intervention}: Max follow-up = {max_followup} months, Mean efficacy = {mean_efficacy:.1f}%")

## 6. Safety Analysis

In [ ]:
# Safety event analysis
print("=== SAFETY ANALYSIS ===")

# Calculate safety event rates
safety_data = df[df['safety_events'].notna()].copy()

if len(safety_data) > 0:
    total_patients_safety = safety_data['n_patients'].sum()
    total_events = safety_data['safety_events'].sum()
    overall_safety_rate = (total_events / total_patients_safety) * 100
    
    print(f"Overall safety profile:")
    print(f"Total patients with safety data: {total_patients_safety}")
    print(f"Total safety events: {total_events}")
    print(f"Safety event rate: {overall_safety_rate:.2f}%")
    
    # Safety by condition
    safety_by_condition = safety_data.groupby('condition').agg({
        'safety_events': 'sum',
        'n_patients': 'sum'
    })
    safety_by_condition['safety_rate'] = (safety_by_condition['safety_events'] / safety_by_condition['n_patients']) * 100
    
    print(f"\nSafety by condition:")
    print(safety_by_condition)
    
    # 95% confidence interval for safety rate
    n = total_patients_safety
    p = total_events / n
    se = np.sqrt(p * (1 - p) / n)
    ci_lower = p - 1.96 * se
    ci_upper = p + 1.96 * se
    
    print(f"\n95% CI for overall safety event rate: {ci_lower*100:.2f}% - {ci_upper*100:.2f}%")
else:
    print("No safety data available for analysis")

## 7. Statistical Power Analysis

In [ ]:
# Power analysis for future trial design
from statsmodels.stats.power import ttest_power

print("=== POWER ANALYSIS FOR FUTURE TRIALS ===")

# Based on observed effect sizes, calculate required sample sizes
epilepsy_effects = epilepsy_data[epilepsy_data['primary_endpoint'] == 'seizure_reduction_percent']['endpoint_value']
if len(epilepsy_effects) > 1:
    mean_effect = epilepsy_effects.mean()
    std_effect = epilepsy_effects.std()
    
    # Calculate Cohen's d (assuming control group has 0% improvement)
    cohens_d = mean_effect / std_effect if std_effect > 0 else 1.0
    
    print(f"Epilepsy trials:")
    print(f"Mean effect size: {mean_effect:.1f}%")
    print(f"Standard deviation: {std_effect:.1f}%")
    print(f"Cohen's d: {cohens_d:.2f}")
    
    # Sample size calculation for 80% power
    for power in [0.8, 0.9]:
        try:
            n_required = ttest_power(effect_size=cohens_d, power=power, alpha=0.05)
            print(f"Sample size needed for {power*100:.0f}% power: {n_required:.0f} per group")
        except:
            print(f"Could not calculate sample size for {power*100:.0f}% power")

# Cost-effectiveness preliminary analysis
print(f"\n=== PRELIMINARY COST-EFFECTIVENESS ===")
# Approximate costs (placeholder values)
cost_per_patient = 150000  # USD, typical stem cell therapy cost
total_patients = df['n_patients'].sum()
estimated_total_cost = total_patients * cost_per_patient

print(f"Estimated research costs to date: ${estimated_total_cost:,.0f}")
print(f"Cost per percentage point of efficacy (epilepsy): ${cost_per_patient / epilepsy_effects.mean():,.0f}" if len(epilepsy_effects) > 0 else "")

## 8. Key Statistical Findings Summary

In [ ]:
# Summary of key statistical findings
print("=== KEY STATISTICAL FINDINGS SUMMARY ===")
print("\n1. EFFICACY ANALYSIS:")

# Epilepsy summary
epilepsy_seizure_reduction = epilepsy_data[epilepsy_data['primary_endpoint'] == 'seizure_reduction_percent']['endpoint_value']
if len(epilepsy_seizure_reduction) > 0:
    print(f"   • Epilepsy seizure reduction: {epilepsy_seizure_reduction.mean():.1f}% ± {epilepsy_seizure_reduction.std():.1f}%")
    print(f"   • Best outcome: {epilepsy_seizure_reduction.max():.1f}% (NRTX-1001 long-term)")

# Diabetes summary
diabetes_insulin_indep = diabetes_t1_data[diabetes_t1_data['primary_endpoint'] == 'insulin_independence_rate']['endpoint_value']
if len(diabetes_insulin_indep) > 0:
    print(f"   • T1DM insulin independence: {diabetes_insulin_indep.mean():.1f}% ± {diabetes_insulin_indep.std():.1f}%")
    print(f"   • Best outcome: {diabetes_insulin_indep.max():.1f}% (VX-880)")

print("\n2. STATISTICAL SIGNIFICANCE:")
# Count trials with significant p-values
significant_trials = df[(df['p_value'].notna()) & (df['p_value'] < 0.05)]
total_trials_with_p = df[df['p_value'].notna()]
print(f"   • Trials with significant results: {len(significant_trials)}/{len(total_trials_with_p)} ({len(significant_trials)/len(total_trials_with_p)*100:.0f}%)" if len(total_trials_with_p) > 0 else "   • Limited p-value data available")

print("\n3. TREATMENT DURABILITY:")
duration_data = df[df['follow_up_months'].notna()]['follow_up_months']
if len(duration_data) > 0:
    print(f"   • Mean follow-up: {duration_data.mean():.1f} months")
    print(f"   • Longest follow-up: {duration_data.max():.0f} months")
    print(f"   • Studies with >12 months follow-up: {len(duration_data[duration_data > 12])}/{len(duration_data)}")

print("\n4. SAFETY PROFILE:")
if len(safety_data) > 0:
    print(f"   • Overall safety event rate: {overall_safety_rate:.2f}%")
    print(f"   • Epilepsy trials: {safety_data[safety_data['condition']=='Epilepsy']['safety_events'].sum()} events in {safety_data[safety_data['condition']=='Epilepsy']['n_patients'].sum()} patients")
    print(f"   • Diabetes trials: {safety_data[safety_data['condition'].str.contains('Diabetes', na=False)]['safety_events'].sum()} events in {safety_data[safety_data['condition'].str.contains('Diabetes', na=False)]['n_patients'].sum()} patients")

print("\n5. RESEARCH GAPS IDENTIFIED:")
print(f"   • Limited long-term data: Only {len(duration_data[duration_data > 24])}/{len(duration_data)} studies >24 months" if len(duration_data) > 0 else "   • Limited duration data")
print(f"   • Small sample sizes: Median N = {df['n_patients'].median():.0f} patients per trial")
print(f"   • Missing confidence intervals: {df['ci_lower'].isna().sum()}/{len(df)} trials")
print(f"   • Limited control groups: {len(df[df['control_group'] > 0])}/{len(df)} trials with controls")